In [5]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# img = mpimg.imread('Kavli-Images\IMG_1223.JPG')  # 替换为你的文件名
# h, w = img.shape[:2]  # 图像的像素高和宽
# # print(img.shape)  # 打印图像的形状

# plt.figure(figsize=(w / 100, h / 100), dpi=100)  # 保持1像素 = 1像素

# plt.imshow(img, interpolation='none')
# plt.axis('off')  # 关闭坐标轴

# plt.show()

In [25]:
%matplotlib qt

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

def read_coords_from_txt(filename):
    coords = []
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            for line in f:
                try:
                    x, y = map(int, line.strip().split(','))
                    coords.append((x, y))
                except:
                    continue
    return coords

def save_coords_to_txt(filename, coords):
    with open(filename, 'w') as f:
        for x, y in coords:
            f.write(f"{x},{y}\n")
    print(f"Saved {len(coords)} coords to {filename}")

def interactive_image_marker(image_path, coord_file):
    img = mpimg.imread(image_path)
    h, w = img.shape[:2]

    initial_coords = read_coords_from_txt(coord_file)

    fig, ax = plt.subplots(figsize=(w / 100, h / 100), dpi=100)
    ax.imshow(img, interpolation='none')
    ax.axis('off')

    marked_points = {}
    point_stack = []
    coords_list = []
    undo_stack = []

    # 初始化已有坐标
    i = 1
    for x, y in initial_coords:
        dot = ax.plot(x, y, 'ro', markersize=4)[0]
        text = ax.text(x + 5, y, i, color='red', fontsize=12)
        marked_points[(x, y)] = (dot, text)
        point_stack.append((x, y))
        coords_list.append((x, y))
        i += 1

    print(f"Loaded {len(initial_coords)} coords from {coord_file}")

    def onclick(event):
        if event.inaxes and event.xdata and event.ydata:
            x, y = int(event.xdata), int(event.ydata)
            key = (x, y)

            if event.button == 1:  # 左键添加
                if key not in marked_points:
                    number = len(marked_points) + 1
                    dot = ax.plot(x, y, 'ro', markersize=4)[0]
                    text = ax.text(x + 5, y, f'{number}', color='red', fontsize=12)
                    marked_points[key] = (dot, text)
                    point_stack.append(key)
                    coords_list.append(key)
                    undo_stack.clear()
                    print(f"Marked {number} at ({x}, {y})")
                else:
                    print(f"Already marked: ({x}, {y})")

            elif event.button == 3:  # 右键点击
                if event.key == 'shift':  # Shift + 右键 => 恢复
                    if undo_stack:
                        key = undo_stack.pop()
                        number = len(marked_points) + 1
                        dot = ax.plot(key[0], key[1], 'ro', markersize=4)[0]
                        text = ax.text(key[0] + 5, key[1], f'{number}', color='red', fontsize=12)
                        marked_points[key] = (dot, text)
                        point_stack.append(key)
                        coords_list.append(key)
                        print(f"Restored: {key}")
                    else:
                        print("Nothing to restore.")
                else:  # 普通右键 => 撤销
                    if point_stack:
                        last_key = point_stack.pop()
                        dot, text = marked_points[last_key]
                        dot.remove()
                        text.remove()
                        del marked_points[last_key]
                        coords_list.remove(last_key)
                        undo_stack.append(last_key)
                        print(f"Removed: {last_key}")
                    else:
                        print("No points to remove.")

            fig.canvas.draw()

    fig.canvas.mpl_connect('button_press_event', onclick)

    def on_close(event):
        save_coords_to_txt(coord_file, coords_list)

    fig.canvas.mpl_connect('close_event', on_close)
    plt.show()


# 使用方法
interactive_image_marker("Kavli-Images\IMG_1223.JPG", coord_file='a.txt')
# interactive_image_marker("Kavli-Images\IMG_1224.JPG", coord_file='b.txt')
# interactive_image_marker("Kavli-Images\IMG_1226.JPG", coord_file='c.txt')


Loaded 15 coords from a.txt


Saved 15 coords to a.txt


In [26]:
import numpy as np

def compute_homography(src_pts, dst_pts):
    assert len(src_pts) == len(dst_pts) and len(src_pts) >= 4, "Need at least 4 point pairs"

    A = []
    for (x, y), (x_prime, y_prime) in zip(src_pts, dst_pts):
        A.append([-x, -y, -1, 0, 0, 0, x_prime*x, x_prime*y, x_prime])
        A.append([0, 0, 0, -x, -y, -1, y_prime*x, y_prime*y, y_prime])

    A = np.array(A)
    U, S, Vt = np.linalg.svd(A)
    h = Vt[-1, :]  # 最后一行对应最小奇异值
    H = h.reshape(3, 3)

    return H / H[2, 2]  # 归一化（最后一个元素设为1）

src = read_coords_from_txt("a.txt")       # 原图点
dst = read_coords_from_txt("b.txt")  # 目标图点

H = compute_homography(src, dst)
print("Estimated H:\n", H)
xx = H@np.append(src[1], 1)
print(xx/xx[2], np.append(dst[1], 1))

Estimated H:
 [[ 2.24588317e+00 -3.91993259e-02 -1.63824161e+03]
 [ 5.26536664e-01  1.57077683e+00 -1.11167794e+03]
 [ 2.41429587e-04 -1.75377106e-05  1.00000000e+00]]
[1.05829606e+03 2.78213610e+03 1.00000000e+00] [1055 2787    1]
